In [3]:
import os
import nltk
import json
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
#from read_dataset import *

%reload_ext autoreload
%autoreload 2

In [4]:
a = "An insulating film substantially free from surface irregularities is RF bias sputtered onto a smooth polycrystalline or micro-roughened surface. Controlled sputtering is performed first at a low reemission coefficient and then, after a substantially continuous layer of insulative amorphous film is deposited over the substrate, increasing the reemission coefficient to a second higher level. A low reemission coefficient is about 0.25 and a high coefficient is about 0.7."
textblober = TextBlob(a)
sentences = textblober.sentences
len(sentences), sentences
nltk.download('punkt')
#nltk.word_tokenize(str(sentences[0]))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# FabNER dataset

In [5]:
def read_files(path):
    data = pd.read_csv(path, sep = ' ', header=None, names = ["tokens","ner_tags"])
    return data

In [37]:
fabner_train = read_files( r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S2-train.txt")
fabner_val = read_files(r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S3-val.txt")
fabner_test = read_files( r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S1-test.txt")
#fabner_train = read_files("fabNER/S2-train.txt") #r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S2-train.txt")
#fabner_val = read_files("fabNER/S2-val.txt") #r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S3-val.txt")
#fabner_test = read_files( "fabNER/S2-test.txt" ) #r"C:\Users\z004r5cc\Documents\Archieve_linuxmachine\instructner0714\instructner\data\fabNER\S1-test.txt")
fabner_total = pd.concat([fabner_train, fabner_val, fabner_test], ignore_index=True)

In [38]:
#fabner_train

#### eliminate prefix, I B O 

In [39]:
def get_jsonl_fabner_simple_with_textblob(df, dataset_path):
    """ 1) transform the csv dataframe to json 
        2) eliminate the prefix B I O 
        3) use TextBlob to separate the sentences"""
    
    with open(dataset_path, 'w', encoding='utf-8') as f:
        mask = df["ner_tags"] != "O"
        df.loc[mask,"ner_tags"] = df[mask]["ner_tags"].apply(lambda x: x.split("-")[1])
        dic_seq = {}
        tokens = ""
        token_list = []
        ner_tags_index = 0
        ner_tags_list = []
        for index, row in df.iterrows():
            # consider all tokens into the string context
            tokens = tokens + row["tokens"] + " " 
            # write all ner_tags in one long list
            ner_tags_list.append(row["ner_tags"])

        #separate to sentences    
        textblober = TextBlob(tokens)
        sentences = textblober.sentences    

        for sentence in sentences:
            token_list = sentence.split(" ")
            dic_seq["tokens"] = token_list
            dic_seq["ner_tags"] = ner_tags_list[ner_tags_index:(ner_tags_index+len(token_list)) ]
            f.write(json.dumps(dic_seq) + "\n")
            # update all parameters
            ner_tags_index = ner_tags_index + len(token_list)
            dic_seq = {}
            token_list = []

In [40]:
get_jsonl_fabner_simple_with_textblob(fabner_train, dataset_path = "fabNER/fabner_simple_train.jsonl")
get_jsonl_fabner_simple_with_textblob(fabner_val, dataset_path = "fabNER/fabner_simple_val.jsonl")
get_jsonl_fabner_simple_with_textblob(fabner_test, dataset_path = "fabNER/fabner_simple_test.jsonl")
get_jsonl_fabner_simple_with_textblob(fabner_total, dataset_path = "fabNER/fabner_simple_total.jsonl")

### get original sentences, transform entites into the chatGPT output format

In [47]:
def get_original_sentences_from_tokens_nertags(path_in, path_out):
    with open(path_in, 'r', encoding='utf-8') as f:
        sentence_list = []
        entities_list = []
        for line in f:
            tokens_ner_tags = json.loads(line)
            sentence = ""
            entity = ""
            for item in tokens_ner_tags["tokens"]:
                sentence = sentence + item + " "
            sentence_list.append(sentence)

    with open(path_out, 'w', encoding='utf-8') as e:                
        for sentence in sentence_list:
            e.write(sentence + '\n')   

def get_entities_from_tokens_nertags(path_in, path_out):
    #read the json file and change to dictionary
    with open(path_in,'r', encoding='utf-8') as f:
        tokens_tags = []
        for line in f:
            token_tag = json.loads(line) 
            tokens_tags.append(token_tag)
    #tokens_tags = tokens_tags[:-1]  
    with open(path_out,'w', encoding='utf-8') as f:
        for line in tokens_tags:
            words = ""
            words_tags_list = ""
            #words_tags_list = []
            for index, tag in enumerate(line["ner_tags"]):
                # take entity, if it's not "O"
                if tag != "O":
                    if index > 0 and line["ner_tags"][index] == line["ner_tags"][index-1]: # index>0 -> index not out of range
                        words = words + " " +  line["tokens"][index]
                    else:
                        words = line["tokens"][index]

                    # the next tag different from the current one        
                    if index < len(line["ner_tags"])-1: # make sure index not out of range
                        if line["ner_tags"][index] != line["ner_tags"][index+1] :
                            words_tags_list =  words_tags_list    +   (words + ": " + line["ner_tags"][index]) + "\n" 
                    
                    # the last token is always full stop sign -> not even enter the if tag != "O":
                    # thus, the next line of code is meaningless
                    else:   
                        words_tags_list = words_tags_list   +   (words + ": " + line["ner_tags"][index])  + "\n"      
            
            if words_tags_list == "":
                words_tags_list = "no entity in this sentence \n"
                
            f.write(  (words_tags_list) + "\n"  )             


#### Get the original sentences

In [42]:
path_in = "fabNER/fabner_simple_train.jsonl"
path_out = "fabNER/fabner_simple_train_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)

path_in = "fabNER/fabner_simple_val.jsonl"
path_out = "fabNER/fabner_simple_val_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)    

path_in = "fabNER/fabner_simple_test.jsonl"
path_out = "fabNER/fabner_simple_test_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)    

path_in = "fabNER/fabner_simple_total.jsonl"
path_out = "fabNER/fabner_simple_total_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)  

#### Get the ner_tags in ChatGPT format

In [48]:
path_in = "fabNER/fabner_simple_train.jsonl"
path_out = "fabNER/fabner_simple_train_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out)  

path_in = "fabNER/fabner_simple_val.jsonl"
path_out = "fabNER/fabner_simple_val_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out)  

path_in = "fabNER/fabner_simple_test.jsonl"
path_out = "fabNER/fabner_simple_test_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out) 

path_in = "fabNER/fabner_simple_total.jsonl"
path_out = "fabNER/fabner_simple_total_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out) 

 #### Remember ro delete the last line of "fabNER/fabner_simple_total_entities.jsonl"

# Assembly dataset - alternators, engines and gearboxes

In [6]:
def read_files_assembly(path):
    data = pd.read_csv(path, sep = '\t', header=None, names = ["tokens","ner_tags"])
    return data 

In [7]:
#assembly_train = read_files_assembly("Assembly-Stanford-NER/dataset/alternators-engines-gearboxes/dataset-A1-E1-G2-train-annotated.tsv")
#assembly_train, assembly_val = train_test_split( assembly_train, test_size=0.2, random_state=0)
#assembly_test = read_files_assembly("Assembly-Stanford-NER/dataset/alternators-engines-gearboxes/dataset-A1-E1-G2-test-annotated.tsv")

assembly_train_aeg = read_files_assembly(r"assembly_dataset/alternators-engines-gearboxes/dataset-A1-E1-G2-train-annotated.tsv")
assembly_test_aeg = read_files_assembly(r"assembly_dataset/alternators-engines-gearboxes/dataset-A1-E1-G2-test-annotated.tsv")
assembly_a2 = read_files_assembly(r"assembly_dataset/alternators/A2-Toyota-Alternator-Build-Manual/Toyota-Alternator-Build-Manual-annotated.tsv")
assembly_e2 = read_files_assembly(r"assembly_dataset/engines/E2-HM1-Instruction-manual/HM1-Instruction-manual-annotated.tsv")
assembly_e3 = read_files_assembly(r"assembly_dataset/engines/E3-SC1A-Instruction-manual/SC1A-Instruction-manual-annotated.tsv")
assembly_e4 = read_files_assembly(r"assembly_dataset/engines/E4-TVR1ABB-Instruction-manual/TVR1ABB-Instruction-manual-annotated.tsv")
assembly_e5 = read_files_assembly(r"assembly_dataset/engines/E5-TVR1A-Instruction-manual/TVR1A-Instruction-manual-annotated.tsv")
assembly_e6 = read_files_assembly(r"assembly_dataset/engines/E6-VR1A-Instruction-manual/VR1A-Instruction-manual-annotated.tsv")
assembly_g2 = read_files_assembly(r"assembly_dataset/gearboxes/G1-Assembly-instructions-for-gearbox/Assembly-instructions-for-gearbox-annotated.tsv")

In [8]:
#assembly_total
#assembly_total = pd.concat([assembly_a2, assembly_e2, assembly_e3, assembly_e4, assembly_e5, assembly_e6, assembly_g2], ignore_index=True)
assembly_total = pd.concat([assembly_train_aeg,assembly_test_aeg], ignore_index=True)

In [9]:
assembly_total

,tokens,ner_tags
0,DISASSEMBLY,OPER
1,AND,O
2,BENCH,O
3,CHECKS,O
4,TEST,O
...,...,...
24342,Looking,O
24343,for,O
24344,any,O
24345,leaks,O


#### transform the pandas dataframe to json

In [17]:
def get_jsonl_assembly(df, dataset_path):
    with open(dataset_path, 'w', encoding='utf-8') as f:
        dic_seq = {}
        token_list = []
        ner_tags_list = []
        for index, row in df.iterrows():
            token_list.append(row["tokens"])
            ner_tags_list.append(row["ner_tags"])
            if row['tokens'] == '0':
                # change "0" to "."
                token_list[-1] = "."
                dic_seq["tokens"] = token_list
                dic_seq["ner_tags"] = ner_tags_list
                f.write(json.dumps(dic_seq) + "\n")
                #json.dump(dic_seq, f, ensure_ascii=False)#, indent=1)
                #print(dic_seq)
                dic_seq = {}
                token_list = []
                ner_tags_list = []   


In [18]:
get_jsonl_assembly(df=assembly_total, dataset_path="assembly_dataset/assembly_total.jsonl")
# row 478, the last 2 entities need to be changed to  ->  "WGT", "WGT", "O"]}

In [51]:
path_in = "assembly_dataset/assembly_total.jsonl"
path_out = "assembly_dataset/assembly_total_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)

path_out = "assembly_dataset/assembly_total_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out) 

 #### Remember ro delete the last line of "assembly_dataset/assembly_total_entities.jsonl"

# Thin film head technology dataset 

In [31]:
def get_jsonl_thin_film_technology_sentence_new_new(path_in, path_out, paragraph = False):

    nltk.download('punkt') 
    with open(path_out, 'w', encoding='utf-8') as e:
        for filename in filter(lambda p: p.endswith("txt"), os.listdir(path_in)):
            #initialization
            files_txt = []
            files_ann = []
            files = {}

            #for txt, get the list for all textual tokens
            filepath = os.path.join(path_in, filename)
            with open(filepath, mode='r',encoding='utf-8') as f:
                files_txt = f.read()   
                f.close()
            files["tokens"] = nltk.word_tokenize(files_txt)

            #for ann, get the correct format for annotated dataset    
            filename_ann = filename[:7] + ".ann"
            filepath_ann = os.path.join(path_in, filename_ann)
            files_ann = pd.read_csv(filepath_ann, sep = '\s+', header=None, 
                                    names = ["T-id","ner_tags","span1", "span2", "tokens1","tokens2","tokens3", "tokens4","tokens5","tokens6",  
                                             "tokens7","tokens8","tokens9","tokens10","tokens11","tokens12","tokens13","tokens14","tokens15"], 
                                    engine='python')
            files_ann = files_ann.replace([None, np.nan], "")
            
            # get the jsonl format that salmon can take as input
            columns_token = ['tokens1','tokens2','tokens3','tokens4','tokens5','tokens6','tokens7','tokens8','tokens9',
                                      "tokens10","tokens11","tokens12","tokens13","tokens14","tokens15"]
            ner_tags = []  
            for index, single_token in enumerate(files['tokens']):
                #for column in columns_token:   
                for i in range(1,15):
                    column = "tokens" + str(i)
                    column2 = "tokens" + str(i+1)
                    for tag_index, row in enumerate(files_ann[column]):
                        row = row.strip(".")
                        if index < len(files["tokens"])-1: ## not reaching the last token of the original sentence 
                            if files_ann[column2][tag_index] != "":
                                if single_token == row and files_ann[column2][tag_index] == files['tokens'][index+1]:
                                    ner_tags.append(files_ann['ner_tags'][tag_index])  # assign the tag to ner_tags according to files_ann`
                                    files_ann[column][tag_index] = "" # after finding the word in the original sentence, set it to '' to avoid double assignment
                                    if ner_tags[-1] == "Others":
                                        ner_tags[-1] = "O" 
                                    break
                            else:  ## the second word of a phase of a certain entity is not empty, 
                                if single_token == row :  #not consider the second word in the condition
                                    ner_tags.append(files_ann['ner_tags'][tag_index])  # assign the tag to ner_tags according to files_ann`
                                    files_ann[column][tag_index] = ""
                                    if ner_tags[-1] == "Others":
                                        ner_tags[-1] = "O" 
                                    break    
                        else: ## have reached the last token of the original sentence 
                            if single_token == row :  # only for the last one
                                ner_tags.append(files_ann['ner_tags'][tag_index])  # assign the tag to ner_tags according to files_ann`
                                files_ann[column][tag_index] = ""
                                if ner_tags[-1] == "Others":
                                    ner_tags[-1] = "O" 
                                break    
                    if len(ner_tags) == index+1:    # if the word is found, do not go though all token columns
                        break                         

                if len(ner_tags) < index+1:    # if no equal string text is found in the columns of tokens
                    ner_tags.append('O')
            files['ner_tags'] = ner_tags

            files = pd.DataFrame(files)
            dic_seq = {}
            token_list = []
            ner_tags_list = []
            if paragraph:
                for index, row in files.iterrows():
                    token_list.append(row["tokens"])
                    ner_tags_list.append(row["ner_tags"])
                dic_seq["tokens"] = token_list
                dic_seq["ner_tags"] = ner_tags_list
                e.write(json.dumps(dic_seq) + "\n")
                dic_seq = {}
                token_list = []
                ner_tags_list = []           
                    
            else:
                for index, row in files.iterrows():
                    token_list.append(row["tokens"])
                    ner_tags_list.append(row["ner_tags"])
                    if row['tokens'] == '.':
                        dic_seq["tokens"] = token_list
                        dic_seq["ner_tags"] = ner_tags_list
                        e.write(json.dumps(dic_seq) + "\n")
                        dic_seq = {}
                        token_list = []
                        ner_tags_list = []         
   

### sentence wise

In [ ]:
get_jsonl_thin_film_technology_sentence_new_new(path_in = "thin-film-technology-dataset", path_out = "thin-film-technology-dataset/thin_film_head_technology_total.jsonl")

In [49]:
path_in = "thin-film-technology-dataset/thin_film_head_technology_total.jsonl"
path_out = "thin-film-technology-dataset/thin_film_head_technology_total_original_sentence.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)

path_out = "thin-film-technology-dataset/thin_film_head_technology_total_entities.jsonl"
get_entities_from_tokens_nertags(path_in, path_out) 

#### Remember ro delete the last line of "thin-film-technology-dataset/thin_film_head_technology_total_entities_paragraph.jsonl"

### paragraph wise

In [34]:
get_jsonl_thin_film_technology_sentence_new_new(path_in = "thin-film-technology-dataset", path_out = "thin-film-technology-dataset/thin_film_head_technology_total_paragraph.jsonl", paragraph = True)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
path_in = "thin-film-technology-dataset/thin_film_head_technology_total_paragraph.jsonl"
path_out = "thin-film-technology-dataset/thin_film_head_technology_total_original_sentence_paragraph.jsonl"
get_original_sentences_from_tokens_nertags(path_in, path_out)

path_out = "thin-film-technology-dataset/thin_film_head_technology_total_entities_paragraph.jsonl"
get_entities_from_tokens_nertags(path_in, path_out) 

 #### Remember ro delete the last line of "thin-film-technology-dataset/thin_film_head_technology_total_entities_paragraph.jsonl"

In [33]:
#get_jsonl_thin_film_technology_sentence_new_new(path_in = "thin-film-technology-dataset/thin-film-small", path_out = "thin-film-technology-dataset/thin-film-small/thin_film_technology_small.jsonl")#, paragraph = True)
#get_jsonl_thin_film_technology_version2_textblob(path_in = "thin-film-technology-dataset/thin-film-small", path_out = "thin-film-technology-dataset/thin-film-small/thin_film_technology_small_textblob.jsonl")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
